In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [5]:
STORM_NUMBER = 24
TITLE = "14-15 March 2021"
SUBTITLE = "8 AM 16 March 2021"
sts = utc(2021, 3, 15, 0)
ets = utc(2021, 3, 22, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [24]:
cull = ['IA-MC-13', 'TLDI4', 602, 547, 691, 656, 730, 816, 'HPTI4', 760, 743, 937, 'SPTI4', 'SXRI4', 'HLSI4', 
        1041, 'IA-ID-4', 875, 812]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    ['RINI4', 6],
    ['IA-BV-1', 4]
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=2, upper=4)
res = mp.postprocess(filename='210316.png')
mp.close()

     state  wfo  val       lon      lat                                geo  \
713     SD  ABR  3.9  -96.5400  44.7300      POINT (273224.397 -24194.207)   
714     AZ  FGZ  3.9 -111.6700  35.1400  POINT (-1060936.086 -1021675.486)   
715     IA  DVN  3.8  -90.7300  42.4100     POINT (759234.612 -244937.769)   
716     AK  AFC  3.8 -149.7300  61.2100   POINT (-2440898.106 2654212.427)   
717     CO  PUB  3.8 -106.1500  38.0900    POINT (-538596.202 -748049.695)   
...    ...  ...  ...       ...      ...                                ...   
1491    CO  BOU  2.0 -104.3280  40.6350    POINT (-365266.623 -475674.572)   
1492    IA  DVN  2.0  -90.2500  41.7800     POINT (806419.700 -310049.639)   
1493    CA  STO  2.0 -120.9475  39.9366   POINT (-1764058.186 -337365.277)   
1494    WI  MKX  2.0  -89.4781  43.0603     POINT (852008.011 -160669.995)   
1495    CO  PUB  2.0 -104.5569  37.1506    POINT (-404572.612 -861138.955)   

      used_for_analysis  nwsli  plotme source  
713            